In [2]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model,Sequential,load_model
from keras.applications.vgg16 import VGG16 ,preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
Image_size=[224,224]

train_path='/content/drive/My Drive/currency_dataset/training'
test_path='/content/drive/My Drive/currency_dataset/validation'

In [5]:
vgg = VGG16(input_shape=Image_size+[3],weights='imagenet',include_top=False)

for layer in vgg.layers:
    layer.trainable=False



x=Flatten()(vgg.output)

first=Dense(2048,activation='relu')(x)
second=Dense(256,activation='relu')(first)
prediction = Dense(11,activation='softmax')(second)

In [6]:
model=Model(inputs=vgg.input,outputs=prediction)

model.summary()

model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [8]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=.2,
                                zoom_range=.1,
                                 horizontal_flip=True
                                )
test_datagen=ImageDataGenerator(rescale=1./255)

In [9]:
train_set=train_datagen.flow_from_directory(train_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_set=test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1301 images belonging to 11 classes.
Found 237 images belonging to 11 classes.


In [10]:
model1=model.fit(train_set,validation_data=test_set,shuffle=True,epochs=20,steps_per_epoch=len(train_set),
  validation_steps=len(test_set))

Epoch 1/20
41/41 [==============================] - 965s 24s/step - loss: 4.2740 - accuracy: 0.2244 - val_loss: 1.5820 - val_accuracy: 0.4515
Epoch 2/20
41/41 [==============================] - 40s 984ms/step - loss: 1.1882 - accuracy: 0.6057 - val_loss: 0.8811 - val_accuracy: 0.7553
Epoch 3/20
41/41 [==============================] - 39s 959ms/step - loss: 0.6619 - accuracy: 0.8109 - val_loss: 0.6537 - val_accuracy: 0.8228
Epoch 4/20
41/41 [==============================] - 40s 970ms/step - loss: 0.4353 - accuracy: 0.8786 - val_loss: 0.4628 - val_accuracy: 0.8903
Epoch 5/20
41/41 [==============================] - 39s 958ms/step - loss: 0.3097 - accuracy: 0.9162 - val_loss: 0.3868 - val_accuracy: 0.9114
Epoch 6/20
41/41 [==============================] - 39s 954ms/step - loss: 0.2130 - accuracy: 0.9470 - val_loss: 0.3757 - val_accuracy: 0.9114
Epoch 7/20
41/41 [==============================] - 39s 950ms/step - loss: 0.2141 - accuracy: 0.9454 - val_loss: 0.3587 - val_accuracy: 0.9325


In [1]:
from keras.preprocessing.image import load_img


def preprocess_img(img):
    
    img=image.load_img(img,target_size=(224,224))
    
    img=image.img_to_array(img)
    
    img=np.expand_dims(img,axis=0)
    
    # normalisation
    
    img=preprocess_input(img)
    
    return img

img=preprocess_img("/content/drive/My Drive/currency_dataset/validation/200/44.jpg")

label=model.predict(img)
print(label)
print(label.argmax())

NameError: ignored

In [12]:
test_set.class_indices

{'1': 0,
 '10': 1,
 '100': 2,
 '1000': 3,
 '2': 4,
 '20': 5,
 '200': 6,
 '2000': 7,
 '5': 8,
 '50': 9,
 '500': 10}

In [64]:
model.save('./currency_recog.h5')


In [2]:
model=load_model('./currency_recog.h5')


NameError: ignored

In [ ]:
model